# Libraries Import

In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import TopSimilarRecommender as TSR
import random

# Datasets

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

# Data Visualization

In [3]:
train.head()

,playlist_id,track_id
0,3271849,2801526
1,5616275,727878
2,11267488,2805283
3,10103900,1515105
4,3836898,2945623


In [4]:
tr_info.head()

,track_id,artist_id,duration,playcount,album,tags
0,2972914,144,224000,49.0,[7],"[54087, 1757, 1718, 116712, 189631]"
1,2750239,246,157000,1.0,[8],"[189631, 3424, 177424, 46208, 205245]"
2,1550729,144,217000,554.0,[9],"[54087, 109806, 46869, 183258, 54337]"
3,2169950,144,207000,200.0,[9],"[54087, 70618, 207003, 109806, 116712]"
4,1903709,144,198000,5.0,[None],"[54087, 81223, 116712, 215342, 71028]"


In [5]:
pl_info.head()

,created_at,playlist_id,title,numtracks,duration,owner
0,1216545588,644838,[12727],27,6522,41504
1,1249326867,7577564,[],9,2650,41504
2,1257766688,3120683,[183],16,3645,44542
3,1248079275,4278112,"[12389, 18698, 18925, 11695, 7117]",15,4151,44542
4,1175201268,8656823,"[12809, 2095, 13257, 12671, 20426, 14448, 18698]",84,18414,44542


In [6]:
tgt_pl.head()

,playlist_id
0,10024884
1,10624787
2,4891851
3,4267369
4,65078


In [7]:
tgt_tr.head()

,track_id
0,1316175
1,3885714
2,3091270
3,226759
4,230596


# Division of the Data in Trainset and Testset

From the target playlists, only the ones with more than 10 elements are kept.

In [8]:
big_target_playlist = [i for i in np.unique(train['playlist_id'].values) if train[train['playlist_id'] == i].shape[0] >= 10]

From the playlists in big_target_playlists, 10 are randomly picked.

In [45]:
test_pl = pd.DataFrame({"playlist_id": list(map(lambda x: random.choice(big_target_playlist), range(100)))})
test_pl

,playlist_id
0,3773307
1,4086673
2,8936042
3,4864550
4,2494890
5,9213
6,3167871
7,3656700
8,5012481
9,3803931


5 elements are removed from each playlist and save in tracks_removed. 

In [79]:
tracks_in_test_pl = pd.merge(train, test_pl, how='inner', on='playlist_id')
tracks_removed= pd.DataFrame(columns=['playlist_id', 'track_id'], dtype='int32')
indexes_to_remove= np.array([], dtype='int32')
for i in np.unique(tracks_in_test_pl['playlist_id'].values):
    #randomly selects 5 elements from playlist i and save indexes
    tmp = (tracks_in_test_pl.where(tracks_in_test_pl['playlist_id']==i).dropna().sample(5)).index
    #insert the items selected in tracks_removed dataFrame
    tracks_removed= tracks_removed.append(tracks_in_test_pl.take(tmp)).astype(int)
    #remove tracks from the original dataframe
    indexes_to_remove=np.append(indexes_to_remove, values=tmp)

tracks_in_train= pd.DataFrame(tracks_in_test_pl)
tracks_in_train.drop(indexes_to_remove, inplace='true')
tracks_removed
5	9213	170092 1781238 597782 852849 3703960

,playlist_id,track_id
984,9213,1246473
913,9213,2515265
911,9213,2001236
956,9213,2746443
917,9213,440165
1462,124371,2053635
1451,124371,2488450
1447,124371,587364
1459,124371,2150352
1455,124371,1253666


Create DataFrame containing the track_ids of the terget track for the test

In [59]:
tgt_test_tracks= pd.DataFrame(tracks_in_test_pl['track_id'])
print(tgt_test_tracks.shape)
tgt_test_tracks = pd.DataFrame.drop_duplicates(tgt_test_tracks)
print(tgt_test_tracks.shape)

(3958, 1)
(3724, 1)


# Recommendation
Building the dictionaries:
fit_dict = {'tracks_info' : tr_info, 
            'attributes' : ['artist_id', 'album', 'tags'],
            'tgt_tracks' : tgt_tr, 
            'n_min_attr' : 2,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}
recommend_dict = {'tgt_playlists' : tgt_pl, #pass the dataset containing only the playlist in the train
                  'train_playlists_tracks_pairs' : train, #pass the dataset without the songs removed for the test
                  'normalize' : False}

In [72]:
fit_dict = {'tracks_info' : tr_info, 
            'attributes' : ['artist_id', 'album', 'tags'],
            'tgt_tracks' : tgt_test_tracks,
            'n_min_attr' : 2,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}

recommend_dict = {'tgt_playlists' : test_pl,
                  'train_playlists_tracks_pairs' : tracks_in_test_pl, 
                  'normalize' : False}

In [73]:
rec = TSR.TopSimilarRecommender()
rec.fit(**fit_dict)
print('Model fitted!')

Fixed dataset
Calculated Indices
Pruning attributes.
Number of attributes before pruning is: 120295
Number of attributes after pruning is: 35250
ICM built
Computed 0 similarities over 100000 with dot measure and 0 shrinkage.
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Computed 1000 similarities over 100000 with dot measure and 0 shrinkage.
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Diagonal to 0
Di

In [80]:
recommendetions = rec.recommend(**recommend_dict)
print('Recommandation done')

Calculated 0 users ratings over 100
URM built
Recommended 0 users over 100
Recommandation done


In [81]:
recommendetions.sort_values('playlist_id')

,playlist_id,track_ids
14,10044718,1134078 1082356 3262479 3028362 1598691
29,10047584,1418528 1097377 1156143 915407 1254581
60,10090783,2907515 2730872 908656 3107123 1081463
33,10132262,2858941 2158480 814822 2156997 2817731
49,10143883,2038075 3659094 3482003 1656677 3738272
15,10183332,641442 3516762 2194711 2299700 2371819
83,10199585,3273559 2667736 193921 2246091 2949017
62,10581762,2590654 1891198 2609171 3522958 199165
24,10732574,424554 3066903 3305608 1839323 2225630
70,10919241,2450413 811570 1681710 3427672 2511556


In [14]:
map_eval = rs.evaluate(recommendetions, tracks_removed, 'MAP')

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [ ]:
tojson('path', [fit_dict, recommend_dict, map_eval])